# Chess Gameplay

First import our `chess_gameplay` module.

In [1]:
import chess_gameplay as chg

## Playing a game

Let's take a look at how games will be played in the tournament. Before running this cell, open the `demo.png` file alongside so you can watch the action!

In [2]:
# Instantiate agents and record team names. Note we're instantiating Agents with no arguments. These agents have been
# passed no models to inform their selections, so they will play purely random moves.

agents = {'white': chg.Agent(), 'black': chg.Agent()}
teams = {'white': 'Team White', 'black': 'Team Black'}

# Then call the `play_game` function. Note that we're playing here to a maximum depth of 5 moves each. In the
# tournament we will be playing to a maximum depth of 50 moves. We are also passing "poseval"=True which means 
# we will use StockFish to evaluate the board state after each move. These evaluations are used to update the 
# eval bar on the left side of the board rendering in `demo.png`. StockFish will be constrained by a time limit
# of 2 seconds and a depth limit of 25.

game_result = chg.play_game(
    agents, 
    teams, 
    max_moves=5, 
    min_seconds_per_move=0, 
    verbose=True, 
    poseval=True, 
    image_path="demo.png"
)

# Run this cell. A file `demo.pgn` will be saved to this repo directory which you can open and watch as it is 
# updated with moves from the game. The game may end in a checkmate, in which case the winner will recieve 1 point 
# and the loser will receieve 0 points. If the game ends in a draw or a stalemate, both will receieve 0 points. If 
# the maximum number of moves is reached without a conclusion to the game, the StockFish evaluations of the final 
# board state are used as the points for each Agent. For each pairing in the tournament, teams will play once as 
# white and once as black. The winner of the pairing will be the team with the highest score summed over the two 
# games. In the event of a draw, the pairing will be played again until a winner is declared.

white: g3
black: Nh6
white: Na3
black: g6
white: e3
black: Ng4
white: h3
black: Bh6
white: d3
black: Nh2
Max moves reached.
White score: 0.523, Black score: 0.477


## Agents using models

Until you have trained a model and saved a checkpoint, you will not be able to run the following cell, but you can see how your model will be called and passed to an Agent to play with.

In [ ]:
import torch
import yaml

# Your model must be imported exactly as follows; from a module called "model" (a file called "model.py") and with
# the class name "Model".

from model import Model

# All necessary arguments for your model to initialize with must be saved in a YAML file called "model_config.yaml"
# so that your model can be instantiated exactly as follows. Your model must NOT require any initialization arguments
# besides those described in your "model_config.yaml" file.

model_config = yaml.safe_load(open("theta_hat_submission/model_config.yaml"))
model = Model(**model_config)

# Your model checkpoint must be called "checkpoint.pt" and must be a dictionary-like object with your model weights
# stored at the key "model" so that it can be loaded into your model exactly as follows.

checkpoint = torch.load("theta_hat_submission/checkpoint.pt", map_location="cpu")
model.load_state_dict(checkpoint["model"])

# Note: when you load your model weights you may see the following warning. You can safely ignore this warning.

ignore = """
/root/.chess/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
"""

In [8]:
from chess.engine import SimpleEngine, Limit
import chess.engine

STOCKFISH_PATH = '/root/chess-hackathon-4/utils/stockfish'

from random import choice, choices
def create_stockfish_agent(elo_rating=1800, time_limit=0.2):
    """Creates an agent that uses Stockfish with a specific ELO rating"""
    class StockfishAgent(chg.Agent):
        def __init__(self):
            super().__init__()
            # Initialize Stockfish engine
            self.engine = SimpleEngine.popen_uci(STOCKFISH_PATH)
            self.engine.configure({"Skill Level": elo_rating // 100})  # Approximate ELO to skill level
            self.time_limit = time_limit
            self.board = chess.Board()  # Initialize a board to track the game state

        def select_move(self, pgn, legal_move_sans):
            # Update board state based on PGN
            self.board = chess.Board()
            if pgn and len(pgn) > 1:  # If there's a PGN history
                moves = pgn.split()
                for move in moves:
                    if '.' not in move:  # Skip move numbers
                        try:
                            self.board.push_san(move)
                        except ValueError:
                            continue

            # Get Stockfish's move recommendation
            result = self.engine.play(self.board, chess.engine.Limit(time=self.time_limit))
            
            # Convert the move to SAN format
            selected_move_san = self.board.san(result.move)
            
            # Verify the move is legal
            if selected_move_san in legal_move_sans:
                return selected_move_san
            else:
                # Fallback to random move if Stockfish suggests an illegal move
                return choice(legal_move_sans)

        def __del__(self):
            if hasattr(self, 'engine'):
                self.engine.quit()

    return StockfishAgent()

In [10]:
# The model is passed as the first positional argument to the Agent, and is then available to the agent to use for
# selecting moves in the game.

# agents = {'white': chg.Agent(model), 'black': chg.Agent(model)}
# teams = {'white': 'Team White', 'black': 'Team Black'}

agents = {'white': chg.Agent(model), 'black': create_stockfish_agent()}
teams = {'white': 'My Model', 'black': 'StockFish Engine'}

# agents = {'white': create_stockfish_agent(), 'black': chg.Agent(model)}
# teams = {'white': 'StockFish Engine', 'black': 'My Model'}


game_result = chg.play_game(
    agents, 
    teams, 
    max_moves=10, 
    min_seconds_per_move=2,
    verbose=True, 
    poseval=True, 
    image_path="demo_2.png"
)

white: g3
black: d5
white: Nf3
black: c6
white: c4
black: g6
white: Bg2
black: d4
white: d3
black: Nf6
white: Bf4
black: Ne4
white: O-O
black: Nd2
white: Nbxd2
black: c5
white: Ne4
black: f6
white: Nxc5
black: Na6
Max moves reached.
White score: 0.535, Black score: 0.465


In [ ]:
# model checkpoints we have so far for a chess vision model:
    
#     model checkpoint 510 : White score: 0.485, Black score: 0.515

#     model checkpoint 490 : White score: 0.499, Black score: 0.501

#     model checkpoint 720 : White score: 0.537, Black score: 0.463
